In [129]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import math

In [123]:
df = pd.read_csv('X_train.csv')

In [124]:
df.head()

,sku,categoryLevel1Id,categoryLevel2Id,brandId,property,userName,reting,date,comment,commentNegative,commentPositive
0,20005023,401,4010201,826,"[{34: 'f982777489055c6563d68c005fd24aad'}, {36...",b2898a81b45310b30beb8fc0c0a9ce1e,2.0,2013-06-28,"2,5 года работала и все...устала! Лампочка гор...",NaN,NaN
1,20020647,403,4030101,1425,"[{34: '9ce895413ebdf6b6dcb69b07dc782591'}, {36...",538c73d64461e13907bb95c51c38bfbc,2.0,2010-07-04,Через 2 месяца после истечении гарантийного ср...,NaN,NaN
2,20020701,401,4010401,124,"[{34: '9ce895413ebdf6b6dcb69b07dc782591'}, {36...",ddca2d0101513a6209db7868eed8be05,4.0,2010-05-27,пользуюсь уже три недели. нареканий ни каких н...,NaN,NaN
3,30012256,203,2030301,93,"[{34: '9ce895413ebdf6b6dcb69b07dc782591'}, {36...",289c20015b3713a82ba5ddf774d996f7,5.0,2016-10-11,Ребят этот системный блок подойдёт для игры кс...,NaN,NaN
4,30011341,205,2050201,656,"[{34: '9ce895413ebdf6b6dcb69b07dc782591'}, {36...",5576f82d149d4f688644fef2322c63ef,5.0,2010-02-26,"я считаю, что яри замечательный телефон! Прият...",NaN,NaN


In [125]:
df = df.drop(['property', 'userName', 'date'], axis=1)

In [126]:
df.head()

,sku,categoryLevel1Id,categoryLevel2Id,brandId,reting,comment,commentNegative,commentPositive
0,20005023,401,4010201,826,2.0,"2,5 года работала и все...устала! Лампочка гор...",NaN,NaN
1,20020647,403,4030101,1425,2.0,Через 2 месяца после истечении гарантийного ср...,NaN,NaN
2,20020701,401,4010401,124,4.0,пользуюсь уже три недели. нареканий ни каких н...,NaN,NaN
3,30012256,203,2030301,93,5.0,Ребят этот системный блок подойдёт для игры кс...,NaN,NaN
4,30011341,205,2050201,656,5.0,"я считаю, что яри замечательный телефон! Прият...",NaN,NaN


In [130]:
def merge_com(data):
    data = data.copy()
    for i in range(data.shape[0]):
        try:
            if math.isnan(data.commentNegative[i]):
                data.loc[i, 'hasNegComment'] = 0
        except TypeError:
            data.loc[i, 'comment'] = data.loc[i, 'comment'] + ' ' + data.loc[i, 'commentNegative']
            data.loc[i, 'hasNegComment'] = 1
       
        try:
            if math.isnan(data.commentPositive[i]):
                data.loc[i, 'hasPosComment'] = 0
        except TypeError:
            data.loc[i, 'comment'] = data.comment[i] + ' ' + data.commentPositive[i]
            data.loc[i, 'hasPosComment'] = 1
       
        data.loc[i, 'n_of_exc'] = data.loc[i, 'comment'].count('!')
        # if i%1000==0:
        #     print(i)
    return data.drop(['commentNegative', 'commentPositive'], axis=1)

In [ ]:
%time 
df = merge_com(df)

In [136]:
import numpy as np
import nltk
from nltk.tokenize import WhitespaceTokenizer
from string import punctuation
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
mystopwords = list(set(stopwords.words('russian')) - set(['не', 'нет', 'более','больше',  'ничего', "но", "хорошо", "лучше"])) + ["х","р", "тыс", "тыщ", "руб"] 

def parse_sentence(sent):
    sent = sent.replace('...', ' ')
    sent = sent.replace('-', ' ')
    sent = sent.replace('/', ' ')
    exclude = set(punctuation + '0123456789'+u'–—'+u'«»')
    merged_text = ''.join(ch for ch in sent if ch not in exclude)
    tokens = WhitespaceTokenizer().tokenize(merged_text.lower())
    tokens = [t_ for t_ in tokens if t_ not in mystopwords]
    tokens = [morph.parse(word)[0].normal_form for word in tokens]
    return np.array(tokens)

def count_exclamation(sent):
    c = 0
    for i in sent:
        if i == '!':
            c += 1
    return c

In [138]:
%time
df['parsed'] = df.comment.apply(parse_sentence)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.2 µs


In [139]:
df.head()

,sku,categoryLevel1Id,categoryLevel2Id,brandId,reting,comment,hasNegComment,hasPosComment,n_of_exc,parsed
0,20005023,401,4010201,826,2.0,"2,5 года работала и все...устала! Лампочка гор...",0.0,0.0,2.0,"[год, работать, устать, лампочка, гореть, боль..."
1,20020647,403,4030101,1425,2.0,Через 2 месяца после истечении гарантийного ср...,0.0,0.0,5.0,"[месяц, истечение, гарантийный, срок, машинка,..."
2,20020701,401,4010401,124,4.0,пользуюсь уже три недели. нареканий ни каких н...,0.0,0.0,0.0,"[пользоваться, неделя, нарекание, какой, не, п..."
3,30012256,203,2030301,93,5.0,Ребят этот системный блок подойдёт для игры кс...,0.0,0.0,0.0,"[ребята, системный, блок, подойти, игра, кс, г..."
4,30011341,205,2050201,656,5.0,"я считаю, что яри замечательный телефон! Прият...",0.0,0.0,12.0,"[считать, ярь, замечательный, телефон, приятно..."


In [140]:
df.to_csv("data_frame.csv", index = False)

In [141]:
df2 = pd.read_csv('data_frame.csv')

In [142]:
df2.head()

,sku,categoryLevel1Id,categoryLevel2Id,brandId,reting,comment,hasNegComment,hasPosComment,n_of_exc,parsed
0,20005023,401,4010201,826,2.0,"2,5 года работала и все...устала! Лампочка гор...",0.0,0.0,2.0,['год' 'работать' 'устать' 'лампочка' 'гореть'...
1,20020647,403,4030101,1425,2.0,Через 2 месяца после истечении гарантийного ср...,0.0,0.0,5.0,['месяц' 'истечение' 'гарантийный' 'срок' 'маш...
2,20020701,401,4010401,124,4.0,пользуюсь уже три недели. нареканий ни каких н...,0.0,0.0,0.0,['пользоваться' 'неделя' 'нарекание' 'какой' '...
3,30012256,203,2030301,93,5.0,Ребят этот системный блок подойдёт для игры кс...,0.0,0.0,0.0,['ребята' 'системный' 'блок' 'подойти' 'игра' ...
4,30011341,205,2050201,656,5.0,"я считаю, что яри замечательный телефон! Прият...",0.0,0.0,12.0,['считать' 'ярь' 'замечательный' 'телефон' 'пр...


In [137]:
len(df.categoryLevel1Id.unique())

48

In [133]:
time?

In [13]:
df.iloc[0]

sku                                                          20005023
categoryLevel1Id                                                  401
categoryLevel2Id                                              4010201
brandId                                                           826
property            [{34: 'f982777489055c6563d68c005fd24aad'}, {36...
userName                             b2898a81b45310b30beb8fc0c0a9ce1e
reting                                                              2
date                                                       2013-06-28
comment             2,5 года работала и все...устала! Лампочка гор...
commentNegative                                                   NaN
commentPositive                                                   NaN
Name: 0, dtype: object

In [14]:
df.iloc[0]['property']

"[{34: 'f982777489055c6563d68c005fd24aad'}, {36: 'ba58121b2b2fb5703c3c3597c35d7e7c'}, {3: 'd09bf41544a3365a46c9077ebb5e35c3'}, {1: '072b030ba126b2f4b2374f342be9ed44'}, {2: 'c0c7c76d30bd3dcaefc96f40275bdc0a'}, {183: 'e04af96afe53462f72f39331b209a810'}, {10880: 'd09bf41544a3365a46c9077ebb5e35c3'}, {10879: 'fc490ca45c00b1249bbe3554a4fdf6fb'}, {115: '320722549d1751cf3f247855f937b982'}, {7853: '3209de58d9d0ec28b1778116d58516d4'}, {7743: '9e9c149882696cd40e2cc4c026804678'}, {353: 'e04af96afe53462f72f39331b209a810'}, {7843: '92fb0c6d1758261f10d052e6e2c1123c'}, {7844: 'b1eec33c726a60554bc78518d5f9b32c'}, {7845: 'e04af96afe53462f72f39331b209a810'}, {7846: 'e04af96afe53462f72f39331b209a810'}, {11093: '46675ef3c0904af8897e7431a759ae80'}, {162: 'd310cb367d993fb6fb584b198a2fd72c'}, {15593: '0c701eb812b630ad4b1de479c5647926'}, {7991: 'e5ddc6b684de0c9d0259a1455760e582'}, {14: '860f03a9d8852be8cf3727694c82847c'}, {342: 'bd12dacbe0580ab0a8842f68fae0a890'}, {468: '12f1d7c953e2e33d273404052b02b9fc'}, {76

In [5]:
set(df.reting)

{1.0,
 1.3,
 1.7,
 2.0,
 2.2999999999999998,
 2.7000000000000002,
 3.0,
 3.2999999999999998,
 3.7000000000000002,
 4.0,
 4.2999999999999998,
 4.7000000000000002,
 5.0}

In [6]:
list(df.reting.unique())

[2.0,
 4.0,
 5.0,
 1.0,
 4.7000000000000002,
 3.0,
 2.2999999999999998,
 2.7000000000000002,
 4.2999999999999998,
 3.2999999999999998,
 1.3,
 1.7,
 3.7000000000000002]

In [7]:
df.reting.value_counts()

5.0    9211
4.0    2609
1.0    1468
3.0    1245
2.0     854
4.7      78
4.3      54
3.7      16
3.3      16
2.7      15
2.3      14
1.3       6
1.7       1
Name: reting, dtype: int64

In [8]:
df.shape

(15587, 11)

In [9]:
df.isnull().sum()

sku                     0
categoryLevel1Id        0
categoryLevel2Id        0
brandId                 0
property                0
userName                0
reting                  0
date                    0
comment                 0
commentNegative     14663
commentPositive     14664
dtype: int64

In [10]:
df[df.commentNegative.notnull()].reting.value_counts()

5.0    521
1.0     90
4.0     83
4.7     63
3.0     41
4.3     40
2.0     28
2.7     15
3.3     14
3.7     13
2.3     12
1.3      3
1.7      1
Name: reting, dtype: int64

In [12]:
df[df.commentNegative.notnull()]

,sku,categoryLevel1Id,categoryLevel2Id,brandId,property,userName,reting,date,comment,commentNegative,commentPositive
13,20003446,410,4100301,48,"[{439: '3cfcdd17dfddb03e17fb5cd09a571502'}, {3...",0d2d4dc69e4d06e919b64d84d76b6381,4.7,2017-01-19,"высокая скорость, мало шума, приемлемая цена, ...","практически нет. Брала для помола специй, в ча...",Пользуюсь данной кофемолкой несколько месяцев ...
47,20002107,405,4050205,980,"[{296: 'ebfe5e1791db03c4cd6ab95801e0977d'}, {1...",fc5f773d2acbfb9c2c05d65bbad3ffd5,5.0,2017-05-15,размер,нет,"купил в выходной, установил за 30 мин, еще чер..."
71,20023389,405,4050204,979,"[{34: '9ce895413ebdf6b6dcb69b07dc782591'}, {36...",6421219d872c945f8aa9dc434c923394,4.0,2017-05-11,"Недорогой бак,от известного бренда и гарантией...",нет за 2 года,Решили с женой поставить водонагреватель в бан...
78,20025773,403,4030101,1425,"[{3: '54229abfcfa5649e7003b83dd4755294'}, {1: ...",8dadaaa46386dbe76a65085e8fb14b2c,5.0,2017-06-28,Стирает отлично. Просто сказка,Вибрация . Сбой программ. Бесконечная стирка н...,1 Ножки стоят отлично. Качания нет.Транспортир...
84,20021413,412,4120201,882,"[{9139: 'e04af96afe53462f72f39331b209a810'}, {...",913dc403bed114644e07ec49c640b015,5.0,2017-03-11,Качество шитья,не выявила,"Всем привет, хочу поделиться с вами моим мнени..."
98,20021837,406,4061001,879,"[{439: '0c701eb812b630ad4b1de479c5647926'}, {1...",7bb1d51e3697a6f9596e9c817c421962,5.0,2017-05-09,"удобство в использовании, тихая работа, качест...",Отсутствуют,"Чайником K711 пользуемся уже третий год, и по..."
104,20022342,406,4060601,62,"[{34: 'f982777489055c6563d68c005fd24aad'}, {36...",bd10fb2ddb09c6c471c0648f49cf8c92,2.3,2016-11-22,нет,очень некачественное антипригарное покрытие ва...,срок владения изделием 3 года. Очень некачеств...
113,20024461,401,4010401,53,"[{34: '9ce895413ebdf6b6dcb69b07dc782591'}, {36...",f53087aea21c57646752c16bf2dc448d,2.3,2017-07-06,Невыявили,"Сильный запах резины, возможно со временем выв...",Нам попалась бракованная-не работало нижнее ко...
127,50032961,517,5170105,360,"[{439: '3cfcdd17dfddb03e17fb5cd09a571502'}, {1...",37339a5698d5e3ea0b50dd53c319f8b4,5.0,2017-05-19,Большой объем. Стильный дизайн. Индикатор заме...,Недостатков нет,Фильтрами пользуюсь постоянно. Этот выглядит о...
131,20023008,401,4010201,709,"[{34: 'f982777489055c6563d68c005fd24aad'}, {36...",3d432e5d4699fbad6bb0322c5b6945a3,5.0,2017-01-21,Отличная вытяжка!круто смотрится в дизайне ква...,не нашла,"Пользуемся месяц, полет прекрасный.Кухня 14 кв..."


In [18]:
"""
pyspelling
Spellchecking for Ukrainian, Russian and English languages.
Used: Python 3, Yandex spellservice.
by proft @ http://proft.me
"""

import os
import requests
from subprocess import Popen, PIPE

ROOT = os.path.abspath(os.path.dirname(__file__))
ICON_OK = os.path.join(ROOT, 'ok.png')
ICON_ERROR = os.path.join(ROOT, 'error.png')
LANG_TRANS = {'ua': 'uk', 'us': 'en', 'ru': 'ru'}


def get_layout():
    """ Get current keyboard layout """

    pipe = Popen("setxkbmap -print | grep xkb_symbols | awk -F'+' '{print $2}'", stdout=PIPE, shell=True)
    layout = pipe.communicate()[0].strip().decode("utf-8")
    return str(layout)


def set_clipboard(text):
    """ Set system clipboard to text """

    xsel_proc = Popen(['xsel', '-bi'], stdin=PIPE)
    xsel_proc.communicate(bytes(text, 'utf-8'))


def get_clipboard():
    """ Get text from system clipboard """

    return os.popen('xsel').read()

if __name__ == '__main__':
    word = get_clipboard()
    params = {'text': get_clipboard(), 'lang': LANG_TRANS[get_layout()]}
    r = requests.get('http://speller.yandex.net/services/spellservice.json/checkText', params=params)

    if r.status_code == 200:
        if len(r.json()) > 0:
            out = r.json()[0]
            variants = [v for v in out['s']]
            set_clipboard(variants[0])
            os.system('notify-send -i %(icon)s "%(caption)s" "%(text)s"' % {
                'icon': ICON_ERROR,
                'caption': word,
                'text': '\n'.join(variants)
            })
        else:
            os.system('notify-send -i %(icon)s "%(text)s"' % {
                'icon': ICON_OK,
                'text': word
            })

NameError: name '__file__' is not defined

In [22]:
from nltk.corpus import stopwords
mystopwords = list(set(stopwords.words('russian')) - set(['не', 'нет', 'ничего', "но", "хорошо", "лучше"]))

for i in stopwords.words('russian'):
    print(i, end=',')

и,в,во,не,что,он,на,я,с,со,как,а,то,все,она,так,его,но,да,ты,к,у,же,вы,за,бы,по,только,ее,мне,было,вот,от,меня,еще,нет,о,из,ему,теперь,когда,даже,ну,вдруг,ли,если,уже,или,ни,быть,был,него,до,вас,нибудь,опять,уж,вам,ведь,там,потом,себя,ничего,ей,может,они,тут,где,есть,надо,ней,для,мы,тебя,их,чем,была,сам,чтоб,без,будто,чего,раз,тоже,себе,под,будет,ж,тогда,кто,этот,того,потому,этого,какой,совсем,ним,здесь,этом,один,почти,мой,тем,чтобы,нее,сейчас,были,куда,зачем,всех,никогда,можно,при,наконец,два,об,другой,хоть,после,над,больше,тот,через,эти,нас,про,всего,них,какая,много,разве,три,эту,моя,впрочем,хорошо,свою,этой,перед,иногда,лучше,чуть,том,нельзя,такой,им,более,всегда,конечно,всю,между,

In [40]:
mystopwords = list(set(stopwords.words('russian')) - set(['не', 'нет', 'ничего', "но", "хорошо", "лучше"]))

In [30]:
#mystopwords

In [ ]:
exclude = set(punctuation + '0123456789'+u'–—'+u'«»')
merged_text = ''.join(ch for ch in merged_text if ch not in exclude)
tokens = WhitespaceTokenizer().tokenize(merged_text.lower())
for i in tokens[:10]: print(i )

In [108]:
import numpy as np
import nltk
from nltk.tokenize import WhitespaceTokenizer
from string import punctuation
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
mystopwords = list(set(stopwords.words('russian')) - set(['не', 'нет', 'более','больше',  'ничего', "но", "хорошо", "лучше"])) + ["х","р", "тыс", "тыщ", "руб"] 

def parse_sentence(sent):
    sent = sent.replace('...', ' ')
    sent = sent.replace('-', ' ')
    sent = sent.replace('/', ' ')
    exclude = set(punctuation + '0123456789'+u'–—'+u'«»')
    merged_text = ''.join(ch for ch in sent if ch not in exclude)
    tokens = WhitespaceTokenizer().tokenize(merged_text.lower())
    tokens = [t_ for t_ in tokens if t_ not in mystopwords]
    tokens = [morph.parse(word)[0].normal_form for word in tokens]
    return np.array(tokens)

def count_exclamation(sent):
    c = 0
    for i in sent:
        if i == '!':
            c += 1
    return c

In [116]:
import time

In [109]:
%time
df['parsed'] = df.comment.apply(parse_sentence)

In [117]:
%time
df.parsed

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 22.9 µs


0        [год, работать, устать, лампочка, гореть, боль...
1        [месяц, истечение, гарантийный, срок, машинка,...
2        [пользоваться, неделя, нарекание, какой, не, п...
3        [ребята, системный, блок, подойти, игра, кс, г...
4        [считать, ярь, замечательный, телефон, приятно...
5        [сегодня, купить, такой, вентилятор, ужасный, ...
6        [привезти, ноут, качество, корпус, дизайн, осо...
7        [купить, увлажнитель, шумный, подсветка, мешат...
8        [комбайн, отличный, пользоваться, год, пользов...
9        [отличный, аппарат, авто, выключение, вообще, ...
10       [смотреть, сегодня, модель, магазин, сначала, ...
11       [пользоваться, большой, год, печь, самый, нача...
12       [пользоваться, жена, несколько, месяц, очень, ...
13       [высокий, скорость, мало, шум, приемлемый, цен...
14       [купить, псп, день, рождениякогда, исполняться...
15       [пользоваться, неделя, нарекание, какой, нет, ...
16       [тот, впарить, шляпа, не, наговариватьофигенны.

In [98]:
count_exclamation(review)

5

In [96]:
np.array(mystopwords)

array(['потом', 'том', 'много', 'уже', 'от', 'все', 'была', 'над', 'бы',
       'куда', 'без', 'этом', 'тут', 'более', 'по', 'вдруг', 'когда',
       'такой', 'про', 'где', 'ним', 'из', 'было', 'мне', 'нее', 'нас',
       'ей', 'наконец', 'ни', 'нельзя', 'ж', 'со', 'опять', 'ее', 'у',
       'какой', 'сейчас', 'потому', 'ты', 'был', 'вот', 'этой', 'за', 'ну',
       'есть', 'как', 'эту', 'между', 'что', 'моя', 'раз', 'больше', 'тот',
       'можно', 'иногда', 'им', 'тебя', 'теперь', 'ли', 'до', 'еще', 'к',
       'при', 'три', 'а', 'нибудь', 'она', 'может', 'мы', 'то', 'вы',
       'вам', 'надо', 'и', 'если', 'себя', 'чего', 'для', 'их', 'этого',
       'него', 'хоть', 'эти', 'себе', 'в', 'два', 'тоже', 'да', 'даже',
       'них', 'только', 'всего', 'после', 'всегда', 'на', 'чуть', 'разве',
       'были', 'с', 'тогда', 'сам', 'тем', 'чтобы', 'или', 'ней', 'зачем',
       'чтоб', 'во', 'о', 'здесь', 'он', 'этот', 'быть', 'ему', 'уж',
       'вас', 'я', 'всю', 'так', 'там', 'мой', 'кто',

In [81]:
review = "Через 2 месяца после истечении гарантийного срока машинка навернулась. Насчитали ремонту на 5500р! Причем брал её итальянской сборки!  Сама машинка была нормальная, правда после стирки снова прополаскивал, но это пожалуй про любую так скажешь - плохо выполаскивается порошок, хотя и сыплю/лью по инструкции и не фигню какую...  А мастер авторизованного сервис-центра поведал мне много интересного... Нет смысла переплачивать сегодня за европейскую страну изготовителя, нет смысла приобретать машинки дороже 12-15 тыс., и что сейчас все производители заведомо делают машинки, чтобы те ломались побыстрее ( в среднем раз в 3-4 года!!!) тем самым стимулируя спрос на новые... Единственное, что еще живо в отношении реального КАЧЕСТВА, так это пожалуй немецкое ""Миеле"" (Бош кстати давно в пролете уже, скатился), но и они конечно тоже ломаются даже при стоимости в 100000... Вывод: Берем стиралку недорогую, тыщ за 8-10, и НЕ СИЛЬНО ОГОРЧАЕМСЯ, когда по истечению 3х-летней самсунговской гарантии вдруг придется ее выбрасывать и снова покупать :)"

In [82]:
review

'Через 2 месяца после истечении гарантийного срока машинка навернулась. Насчитали ремонту на 5500р! Причем брал её итальянской сборки!  Сама машинка была нормальная, правда после стирки снова прополаскивал, но это пожалуй про любую так скажешь - плохо выполаскивается порошок, хотя и сыплю/лью по инструкции и не фигню какую...  А мастер авторизованного сервис-центра поведал мне много интересного... Нет смысла переплачивать сегодня за европейскую страну изготовителя, нет смысла приобретать машинки дороже 12-15 тыс., и что сейчас все производители заведомо делают машинки, чтобы те ломались побыстрее ( в среднем раз в 3-4 года!!!) тем самым стимулируя спрос на новые... Единственное, что еще живо в отношении реального КАЧЕСТВА, так это пожалуй немецкое Миеле (Бош кстати давно в пролете уже, скатился), но и они конечно тоже ломаются даже при стоимости в 100000... Вывод: Берем стиралку недорогую, тыщ за 8-10, и НЕ СИЛЬНО ОГОРЧАЕМСЯ, когда по истечению 3х-летней самсунговской гарантии вдруг пр

In [83]:
' '.join( s for s in parse_sentence(review))

'месяца истечении гарантийного срока машинка навернулась насчитали ремонту причем брал её итальянской сборки сама машинка нормальная правда стирки снова прополаскивал но это пожалуй любую скажешь плохо выполаскивается порошок хотя сыплюлью инструкции не фигню какую мастер авторизованного сервисцентра поведал интересного нет смысла переплачивать сегодня европейскую страну изготовителя нет смысла приобретать машинки дороже производители заведомо делают машинки те ломались побыстрее среднем года самым стимулируя спрос новые единственное живо отношении реального качества это пожалуй немецкое миеле бош кстати давно пролете скатился но ломаются стоимости вывод берем стиралку недорогую не сильно огорчаемся истечению хлетней самсунговской гарантии придется выбрасывать снова покупать'

In [92]:
parse_sentence(review).shape

(96,)

In [107]:
review.replace('...', ' ')
review.replace('-', ' ')
review.replace('/', ' ')

'Через 2 месяца после истечении гарантийного срока машинка навернулась. Насчитали ремонту на 5500р! Причем брал её итальянской сборки!  Сама машинка была нормальная, правда после стирки снова прополаскивал, но это пожалуй про любую так скажешь - плохо выполаскивается порошок, хотя и сыплю лью по инструкции и не фигню какую...  А мастер авторизованного сервис-центра поведал мне много интересного... Нет смысла переплачивать сегодня за европейскую страну изготовителя, нет смысла приобретать машинки дороже 12-15 тыс., и что сейчас все производители заведомо делают машинки, чтобы те ломались побыстрее ( в среднем раз в 3-4 года!!!) тем самым стимулируя спрос на новые... Единственное, что еще живо в отношении реального КАЧЕСТВА, так это пожалуй немецкое Миеле (Бош кстати давно в пролете уже, скатился), но и они конечно тоже ломаются даже при стоимости в 100000... Вывод: Берем стиралку недорогую, тыщ за 8-10, и НЕ СИЛЬНО ОГОРЧАЕМСЯ, когда по истечению 3х-летней самсунговской гарантии вдруг пр

In [94]:
parse_sentence(review)

array(['месяца', 'истечении', 'гарантийного', 'срока', 'машинка',
       'навернулась', 'насчитали', 'ремонту', 'причем', 'брал', 'её',
       'итальянской', 'сборки', 'сама', 'машинка', 'нормальная', 'правда',
       'стирки', 'снова', 'прополаскивал', 'но', 'это', 'пожалуй', 'любую',
       'скажешь', 'плохо', 'выполаскивается', 'порошок', 'хотя',
       'сыплюлью', 'инструкции', 'не', 'фигню', 'какую', 'мастер',
       'авторизованного', 'сервисцентра', 'поведал', 'интересного', 'нет',
       'смысла', 'переплачивать', 'сегодня', 'европейскую', 'страну',
       'изготовителя', 'нет', 'смысла', 'приобретать', 'машинки', 'дороже',
       'производители', 'заведомо', 'делают', 'машинки', 'те', 'ломались',
       'побыстрее', 'среднем', 'года', 'самым', 'стимулируя', 'спрос',
       'новые', 'единственное', 'живо', 'отношении', 'реального',
       'качества', 'это', 'пожалуй', 'немецкое', 'миеле', 'бош', 'кстати',
       'давно', 'пролете', 'скатился', 'но', 'ломаются', 'стоимости',
   

In [87]:
len(a)

96

In [88]:
a.reshape()

TypeError: function takes exactly 1 argument (0 given)